In [1]:
pip install nba_api selenium pandas


Note: you may need to restart the kernel to use updated packages.


In [1]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster
import pandas as pd
import time

def get_player_team_pairs(start_year=2010, end_year=2025):
    nba_teams = teams.get_teams()
    player_team_set = set()
    player_team_list = []

    for year in range(start_year, end_year + 1):
        season = f"{year}-{str(year + 1)[-2:]}"
        print(f"Processing season: {season}")
        for team in nba_teams:
            try:
                roster = commonteamroster.CommonTeamRoster(season=season, team_id=team['id'])
                df = roster.get_data_frames()[0]
                for _, row in df.iterrows():
                    key = (row['PLAYER_ID'], team['nickname'])
                    if key not in player_team_set:
                        player_team_set.add(key)
                        player_team_list.append({
                            'PLAYER_NAME': row['PLAYER'],
                            'PLAYER_ID': row['PLAYER_ID'],
                            'TEAM': team['nickname']
                        })
                time.sleep(0.6)
            except Exception as e:
                print(f"Error for {team['nickname']} in {season}: {e}")

    return pd.DataFrame(player_team_list)


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument("--start-maximized")
    #driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    #options.add_argument("--headless")  # Optional: show browser if removed
    return webdriver.Chrome(options=options)

# def wait_for_table(driver):
#     WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.CSS_SELECTOR, "table.datatable.center"))
#     )
def wait_for_table_or_no_results(driver):
    WebDriverWait(driver, 10).until(
        EC.any_of(
            EC.presence_of_element_located((By.CSS_SELECTOR, "table.datatable.center")),
            EC.presence_of_element_located((By.XPATH, "//p[@class='bodyCopy' and contains(text(), 'There were no matching transactions found.')]"))
        )
    )


def check_no_results_page(driver):
    try:
        element = driver.find_element(By.XPATH, "//p[@class='bodyCopy' and contains(text(), 'There were no matching transactions found.')]")
        return True
    except:
        return False

def search_player_injuries(driver, player_name, team_name):
    try:
        #driver.get("https://www.prosportstransactions.com/basketball/Search/Search.php")  # 🔁 Replace with actual URL
        #time.sleep(2)

        ### Player name input
        player_input = driver.find_element(By.ID, "PlayerName2")  # 🔁
        player_input.clear()
        player_input.send_keys(player_name)
        #time.sleep(0.5)

        ### Team nickname input
        team_input = driver.find_element(By.ID, "Team")  # 🔁
        team_input.clear()
        team_input.send_keys(team_name)
        #time.sleep(0.5)

        ### Year filters
        start_year = driver.find_element(By.ID, "BeginDate")  # 🔁
        start_year.clear()
        start_year.send_keys("2010-10-01")  # Start date

        end_year = driver.find_element(By.ID, "EndDate")  # 🔁
        end_year.clear()
        end_year.send_keys("2025-05-31")  # End date
        #time.sleep(0.5)

        ### 🔘 Toggle extra options before search
        # Example checkboxes or buttons
        # Replace these with actual IDs/classes/xpaths
        try:
            # Uncheck first box if checked
            movement_box = driver.find_element(By.ID, "PlayerMovementChkBx")
            if movement_box.is_selected():
                movement_box.click()

            # Check the next 5 boxes if not already checked
            for checkbox_id in [
                "ILChkBx", "NBADLChkBx", "InjuriesChkBx", "PersonalChkBx", "DisciplinaryChkBx"
            ]:
                box = driver.find_element(By.ID, checkbox_id)
                if not box.is_selected():
                    box.click()

            #time.sleep(0.5)

        except Exception as opt_err:
            print(f"Warning: couldn't toggle one or more options: {opt_err}")



        ### Search button
        driver.find_element(By.ID, "submit").click()

        try:
            wait_for_table_or_no_results(driver)

            if check_no_results_page(driver):
                return [0]
        except Exception:
            print(f"🔄 No table loaded for {player_name} ({team_name}), retrying submit...")
            time.sleep(2)
            driver.find_element(By.ID, "submit").click()
            try:
                wait_for_table_or_no_results(driver)

                if check_no_results_page(driver):
                    return [0]
            except:
                print(f"❌ Still no results after retry for {player_name} ({team_name})")
                return []


        ### Parse results table
        result_data = []

        while True:
            # --- Extract table rows on this page ---
            try:
                table = driver.find_element(By.CSS_SELECTOR, "body > div.container > table.datatable.center")
                rows = table.find_elements(By.TAG_NAME, "tr")[1:]  # Skip header row
                for row in rows:
                    cols = row.find_elements(By.TAG_NAME, "td")
                    row_data = [col.text.strip() for col in cols]
                    if row_data:
                        result_data.append([player_name, team_name] + row_data)
            except:
                break
            # --- Check for "Next" button ---
            try:
                pagination_td = driver.find_element(By.XPATH, "//p[@class='bodyCopy' and contains(., 'Next')]")
                next_link = pagination_td.find_element(By.TAG_NAME, "a")
                next_link.click()
                time.sleep(1)
            except:
                break  # Either "Next" is not present or not a link anymore

            # try:
            #     pagination_td = driver.find_element(By.XPATH, "//p[@class='bodyCopy']")
            #     next_links = pagination_td.find_elements(By.TAG_NAME, "a")
            #     found_next = False
            #     for link in next_links:
            #         if "Next" in link.text:
            #             link.click()
            #             wait_for_table_or_no_results(driver)
            #             found_next = True
            #             break
            #     if not found_next:
            #         break
            # except:
            #     break



        return result_data

    except Exception as e:
        print(f"Error scraping {player_name} ({team_name}): {e}")
        return []


In [23]:
player_name = "LeBron James"
team_name = "Lakers"
driver = setup_driver()
driver.implicitly_wait(10)  # Wait for elements to load
injury_rows = search_player_injuries(driver, player_name, team_name)


In [ ]:
injury_rows

In [3]:
player_team_df = get_player_team_pairs()

Processing season: 2010-11
Processing season: 2011-12
Processing season: 2012-13
Processing season: 2013-14
Processing season: 2014-15
Processing season: 2015-16
Processing season: 2016-17
Processing season: 2017-18
Processing season: 2018-19
Processing season: 2019-20
Processing season: 2020-21
Processing season: 2021-22
Processing season: 2022-23
Processing season: 2023-24
Processing season: 2024-25
Processing season: 2025-26


In [4]:
player_team_df.to_csv("player_team_pairs.csv", index=False)

In [5]:
unique = player_team_df['PLAYER_NAME'].unique()

In [ ]:
unique

In [15]:
import pandas as pd
import csv
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# Load your player-team DataFrame
# Or use your existing in-memory player_team_df
player_team_df = pd.read_csv("data/trail_blazers_pairs.csv")

done_set = set()
try:
    with open("scraped_injuries.csv", newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            # if (row[0] == 'Brandon Goodwin') & (row[1] == 'Hawks'):
            #     break
            done_set.add((row[0], row[1]))
except FileNotFoundError:
    pass

try:
    with open("no_data.csv", newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            done_set.add((row[0], row[1]))
except FileNotFoundError:
    pass

csv_file = open("scraped_injuries.csv", "a", newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
if csv_file.tell() == 0:
    csv_writer.writerow(["PLAYER_NAME", "TEAM", "Date", "Team", "Acquired", "Relinquished", "Notes"])

no_data_file = open("no_data.csv", "a", newline='', encoding='utf-8')
no_data_writer = csv.writer(no_data_file)
if no_data_file.tell() == 0:
    no_data_writer.writerow(["PLAYER_NAME", "TEAM"])

# Initial load with 10s wait for manual verification
driver = setup_driver()
driver.get("https://www.prosportstransactions.com/basketball/Search/Search.php")
print("🕒 Waiting 10s for manual verification...")
time.sleep(10)

for _, row in player_team_df.iterrows():
    player_name = row['PLAYER_NAME']
    team_name = row['TEAM']

    if (player_name, team_name) in done_set:
        continue

    print(f"Scraping: {player_name} ({team_name})")

    try:
        injury_rows = search_player_injuries(driver, player_name, team_name)
        if injury_rows == [0]:
            print(f"⚠️ No results for {player_name} ({team_name})")
            no_data_writer.writerow([player_name, team_name])
        elif injury_rows:
            csv_writer.writerows(injury_rows)
        done_set.add((player_name, team_name))
    except Exception as e:
        print(f"❌ Error on {player_name} ({team_name}): {e}")


    print(f"🔁 Returning to form...")
    try:
        driver.back()
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "PlayerName2"))
        )
    except Exception as e:
        print(f"🔁 Failed back navigation: {e}, reloading form...")
        driver.get("https://www.prosportstransactions.com/basketball/Search/Search.php")
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "PlayerName2"))
        )

csv_file.close()
no_data_file.close()
driver.quit()



🕒 Waiting 10s for manual verification...
Scraping: Armon Johnson (Blazers)
🔁 Returning to form...
Scraping: Wesley Matthews (Blazers)
🔁 Returning to form...
Scraping: Gerald Wallace (Blazers)
🔁 Returning to form...
Scraping: Rudy Fernandez (Blazers)
🔁 Returning to form...
Scraping: Brandon Roy (Blazers)
🔁 Returning to form...
Scraping: Patty Mills (Blazers)
🔁 Returning to form...
Scraping: Elliot Williams (Blazers)
🔁 Returning to form...
Scraping: Luke Babbitt (Blazers)
🔁 Returning to form...
Scraping: LaMarcus Aldridge (Blazers)
🔁 Returning to form...
🔁 Failed back navigation: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff7ba36cda5+78885]
	GetHandleVerifier [0x0x7ff7ba36ce00+78976]
	(No symbol) [0x0x7ff7ba129bca]
	(No symbol) [0x0x7ff7ba180766]
	(No symbol) [0x0x7ff7ba180a1c]
	(No symbol) [0x0x7ff7ba1d4467]
	(No symbol) [0x0x7ff7ba1a8bcf]
	(No symbol) [0x0x7ff7ba1d122f]
	(No symbol) [0x0x7ff7ba1a8963]
	(No symbol) [0x0x7ff7ba1716b1]
	(No symbol) [0x0x7ff7ba172443]
	GetHandleVerifie

In [13]:
player_team_df_2013 = get_player_team_pairs(2013,2013)
player_team_df_2013.to_csv("data/player_team_pairs_2013.csv", index=False)

Processing season: 2013-14


In [17]:
player_team_df_2014 = get_player_team_pairs(2014,2014)
player_team_df_2015 = get_player_team_pairs(2015,2015)
player_team_df_2016 = get_player_team_pairs(2016,2016)
player_team_df_2017 = get_player_team_pairs(2017,2017)
player_team_df_2018 = get_player_team_pairs(2018,2018)
player_team_df_2019 = get_player_team_pairs(2019,2019)
player_team_df_2020 = get_player_team_pairs(2020,2020)
player_team_df_2021 = get_player_team_pairs(2021,2021)
player_team_df_2022 = get_player_team_pairs(2022,2022)
player_team_df_2023 = get_player_team_pairs(2023,2023)
player_team_df_2024 = get_player_team_pairs(2024,2024)
player_team_df_2014.to_csv("data/player_team_pairs_2014.csv", index=False)
player_team_df_2015.to_csv("data/player_team_pairs_2015.csv", index=False)
player_team_df_2016.to_csv("data/player_team_pairs_2016.csv", index=False)
player_team_df_2017.to_csv("data/player_team_pairs_2017.csv", index=False)
player_team_df_2018.to_csv("data/player_team_pairs_2018.csv", index=False)
player_team_df_2019.to_csv("data/player_team_pairs_2019.csv", index=False)
player_team_df_2020.to_csv("data/player_team_pairs_2020.csv", index=False)
player_team_df_2021.to_csv("data/player_team_pairs_2021.csv", index=False)
player_team_df_2022.to_csv("data/player_team_pairs_2022.csv", index=False)
player_team_df_2023.to_csv("data/player_team_pairs_2023.csv", index=False)
player_team_df_2024.to_csv("data/player_team_pairs_2024.csv", index=False)

Processing season: 2014-15
Processing season: 2015-16
Processing season: 2016-17
Processing season: 2017-18
Processing season: 2018-19
Processing season: 2019-20
Processing season: 2020-21
Processing season: 2021-22
Processing season: 2022-23
Processing season: 2023-24
Processing season: 2024-25


In [5]:
injuries_df = pd.read_csv("scraped_injuries.csv")

In [6]:
unique_teams = injuries_df['TEAM'].unique()

In [ ]:
unique_teams

In [13]:
trail_blazers_pairs = pd.read_csv("player_team_pairs.csv")
trail_blazers_pairs = trail_blazers_pairs[trail_blazers_pairs['TEAM'] == 'Trail Blazers']
trail_blazers_pairs['TEAM'] = 'Blazers'
trail_blazers_pairs.to_csv("data/trail_blazers_pairs.csv", index=False)


In [ ]:
trail_blazers_pairs

In [16]:
injuries_df = pd.read_csv("scraped_injuries.csv")

In [17]:
injuries_notes_list = injuries_df['Notes'].unique().tolist()

In [21]:
injuries_notes_df = pd.DataFrame(injuries_notes_list, columns=['Notes'])
injuries_notes_df.to_csv("data/injuries_notes.csv", index=False)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import pandas as pd

# Vectorize notes
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_features=1000)
X = vectorizer.fit_transform(injuries_notes_df["Notes"].astype(str))

# Try clustering
k = 10  # Start with 10 clusters, we’ll tune this
kmeans = KMeans(n_clusters=k, random_state=42)
injuries_notes_df["cluster"] = kmeans.fit_predict(X)


c:\Users\yuval\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [23]:
# See top notes in each cluster
for i in range(k):
    print(f"\n--- Cluster {i} ---")
    print(injuries_notes_df[injuries_notes_df["cluster"] == i]["Notes"].head(10).to_string(index=False))



--- Cluster 0 ---
arthroscopic surgery on right elbow to remove l...
       surgery on left shoulder (out indefinitely)
surgery to repair torn left pectoral muscle (ou...
surgery to repair torn pectoral muscle (out for...
surgery on back to repair bulging disc (out ind...
         surgery on left shoulder (out for season)
arthroscopic surgery on left wrist (out for sea...
surgery on heart to repair aortic aneurysm (out...
surgery on left knee to repair torn ACL (out fo...
recovering from surgery on left knee to repair ...

--- Cluster 1 ---
placed on IL with left shoulder injury (out for...
               placed on IL with right knee injury
                placed on IL with left knee injury
placed on IL with right ankle injury (out for s...
placed on IL with left Achilles injury (out for...
           placed on IL with right shoulder injury
                     placed on IL with knee injury
placed on IL with left ankle injury (out for se...
   placed on IL with groin injury (out for s

In [25]:
import pandas as pd

# Load notes
df = pd.read_csv("data/injuries_notes.csv")  # replace with your correct path
df.columns = ["Notes"]

# Drop missing/duplicate
df = df.dropna().drop_duplicates().reset_index(drop=True)

# Sample a small subset for manual labeling
manual_sample = df.sample(n=120, random_state=42).copy()

# Add a new column for manual labels (fill this manually)
manual_sample["manual_label"] = ""

# Save to CSV to label manually in Excel / Google Sheets
manual_sample.to_csv("data/notes_to_label.csv", index=False)


In [39]:
# Load your manually labeled file
labeled_df = pd.read_csv("data/notes_to_label.csv")
labeled_df = labeled_df.dropna(subset=["manual_label"])  # remove rows not labeled

# Features and labels
X = labeled_df["Notes"]
y = labeled_df["manual_label"]


In [40]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# TF-IDF Vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2), stop_words="english", max_features=1000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Logistic Regression classifier
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_vec, y_train)

# Evaluate
y_pred = clf.predict(X_test_vec)
print("Classification Report:\n", classification_report(y_test, y_pred))


Classification Report:
                         precision    recall  f1-score   support

               DTD/DNP       1.00      1.00      1.00         6
    G-leaue assignment       1.00      0.50      0.67         2
called up from G-leaue       1.00      1.00      1.00         1
                 fined       1.00      1.00      1.00         2
               illness       0.71      0.71      0.71         7
                injury       0.62      0.89      0.73         9
      long term injury       0.78      1.00      0.88         7
         other reasons       1.00      0.57      0.73         7
              returned       1.00      1.00      1.00         3
              soreness       1.00      0.75      0.86         4
               surgery       1.00      0.67      0.80         3
            suspension       1.00      1.00      1.00         3

              accuracy                           0.83        54
             macro avg       0.93      0.84      0.86        54
          weig

In [41]:
label_counts = y.value_counts()
label_counts

manual_label
injury                    44
long term injury          35
illness                   35
other reasons             32
DTD/DNP                   30
soreness                  21
surgery                   17
suspension                16
returned                  14
fined                     11
G-leaue assignment         8
called up from G-leaue     6
Name: count, dtype: int64

In [44]:
X_all = vectorizer.transform(df["Notes"])
df["predicted_label"] = clf.predict(X_all)
df["confidence"] = clf.predict_proba(X_all).max(axis=1)
df.to_csv("data/all_injury_notes_labeled.csv", index=False)


In [46]:
X_all = vectorizer.transform(df["Notes"])
probs = clf.predict_proba(X_all)

import numpy as np

# Get indices of top 2 predicted classes for each note
top2_indices = np.argsort(probs, axis=1)[:, -2:][:, ::-1]  # reverse to get top 2 in order

# Map back to class names
top1_labels = clf.classes_[top2_indices[:, 0]]
top2_labels = clf.classes_[top2_indices[:, 1]]
top1_probs = probs[np.arange(len(probs)), top2_indices[:, 0]]
top2_probs = probs[np.arange(len(probs)), top2_indices[:, 1]]

# Add to DataFrame
df["top1_label"] = top1_labels
df["top1_prob"] = top1_probs
df["top2_label"] = top2_labels
df["top2_prob"] = top2_probs

df.to_csv("data/injury_notes_with_top2_predictions.csv", index=False)
